# Пользовательские демографические группы
Ноутбук с примерами формирования пользовательских демографических групп.

Общие принципы формирования демографических групп:
- формируются на основе значений каталога социально-демографических переменных
- доступные операторы:
    ```
    = (равенство)
    != (неравенство)
    > (больше)
    < (меньше)
    AND (И)
    OR (ИЛИ)
    IN() (вхождение в список значений)
    NOTIN() (невхождение в список значений)
    ```

Рассмотрим подробнее на примере следующих демографических групп:
1. Мужчины 35-44 лет
2. Мужчины 18-24 лет или женщины 25-34 лет
3. Все 20-56 лет
4. 25-54 не руководители
5. Жители городов России 700 000+ со средним уровнем дохода или выше среднего (BC)

## Инициализация
Загрузим библиотеки, которые нам помогут обращаться к API Responsum и работать с данными

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
from pathlib import Path
from IPython.display import JSON

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

In [ ]:
# зададим тип установки
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создадим объекты для работы с каталогами и заданиями,
# а так же загрузим каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

## Список социально-демографических переменных
Для формирования синтаксиса демографических групп нам понадобится каталог социально-демографических переменных.

Загрузим каталог (подробнее работа с каталогами описана в [ноутбуке](catalogs.ipynb#catalogs_site_avito))

In [ ]:
demo_cat = rcats.get_demo()
demo_cat

Для удобства можно выгрузить каталог в Excel и работать с ним там. Файл сохраняется в директорию с ноутбуком.

In [ ]:
writer = pd.ExcelWriter('demo_cat.xlsx')
demo_cat.to_excel(writer, 'demo_cat', index=False)
writer.save()

### Расчет аудитории Интернета

Расчет данных по всем проектам займет достаточно много времени у Responsum, поэтому воспользуемся технической страницей WEB-Index, в которой учитывается вся аудитория.
Техническая страница WEB-Index __site_id = 101__

Важно учитывать в случае расчета статистики OTS:
- если считать с ограничением по Технической странице в медиа-фильтре, то полученная статистика OTS будет показывать общее количество заходов в Интернет за указанный период;
- если считать без ограничений в медиа-фильтре (media_filter = None), то полученная статистика OTS будет показывать общее количество загрузок страниц интернет-проектов, содержащихся в вашем медиа-каталоге, за указанный период;
- статистика OTS корректна для работы с десктопными данными и данными по рекламным кампаниям.


## 1. Мужчины 35-44 лет

Из каталога видим, что для отбора мужчин нам нужно значение 1 переменной SEX
```
    SEX = 1
```
Возрастной группе 35-44 соответствует значение 4 переменной AGE_GROUPS
```
    AGE_GROUPS = 4
```
Объединяем выражения через оператор AND
```
    SEX = 1 AND AGE_GROUPS = 4
```

### Задаем параметры расчета

In [ ]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Формируем задание на расчет

В demo_filter передаем сформированный синтаксиc переменной в кавычках

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'M35-44'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = 'site = 101'

# задаем социально-демографическую группу в demo_filter
demo_filter = "SEX = 1 AND AGE_GROUPS = 4"

# задаем список статистик для расчета
statistics = ["UnwReach"]

# указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
    }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

## 2. Мужчины 18-24 лет или женщины 25-34 лет

Возвращаемся к каталогу социально-демографических переменных. Для отбора мужчин 18-24 лет нам нужно значение 1 переменной SEX и значение 2 переменной AGE_GROUPS
```
SEX = 1 AND AGE_GROUPS = 2
```
Для отбора женщин 25-34 лет синтаксис будет следующим:
```
SEX = 2 AND AGE_GROUPS = 3
```
Объединяем через оператор OR
```
(SEX = 1 AND AGE_GROUPS = 2) OR (SEX = 2 AND AGE_GROUPS = 3)
```

### Задаем параметры расчета

In [ ]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Формируем задание на расчет

В demo_filter передаем сформированный синтаксиc переменной в кавычках

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'M18-24_F25-34'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = 'site = 101'

# задаем социально-демографическую группу в demo_filter
demo_filter = "(SEX = 1 AND AGE_GROUPS = 2) OR (SEX = 2 AND AGE_GROUPS = 3)"

# задаем список статистик для расчета
statistics = ["UnwReach"]

# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
    }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

## 3. Все 20-56 лет

Возраст 20-56 лет не получится собрать из переменной AGE_GROUPS, нам понадобится переменная AGE.

Зададим диапазон возраста.
Можно использовать только операторы строгого сравнения "больше"(>) и "меньше"(<), поэтому наше выражение будет выглядеть так:
```
AGE > 19 AND AGE < 57
```

### Задаем параметры расчета

In [ ]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# Проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Формируем задание на расчет

В demo_filter передаем сформированный синтаксиc переменной в кавычках

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'All20-56'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = 'site = 101'

# задаем социально-демографическую группу в demo_filter
demo_filter = "(AGE > 19 AND AGE < 57)"

# задаем список статистик для расчета
statistics = ["UnwReach"]

# указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
}

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

## 4. 25-54 не руководители

Для отбора возрастной группы 25-54 нам понадобятся значения 3, 4 и 5 переменной AGE_GROUPS. Для упрощения мы не будем составлять три условия через OR, а воспользуемся оператором IN
```
AGE_GROUPS IN (3, 4, 5)
```
Условие "не руководители" можно также коротко задать при помощи оператора неравенства "!="
```
OCCUPATION2 != 1
```
Объединяем через оператор AND
```
AGE_GROUPS IN (3, 4, 5) AND OCCUPATION2 != 1
```

### Задаем параметры расчета

In [ ]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Формируем задание на расчет

В demo_filter передаем сформированный синтаксиc переменной в кавычках

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = '25-54_not_dir'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = 'site = 101'

# задаем социально-демографическую группу в demo_filter
demo_filter = "AGE_GROUPS IN (3, 4, 5) AND OCCUPATION2 != 1"

# задаем список статистик для расчета
statistics = ["UnwReach"]

# указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
    }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

## 5. Жители городов России 700 000+ со средним уровнем дохода или выше среднего (BC)

Численность населения городов России 700 000+ зададим при помощи переменной CITY_TYPE2
```
CITY_TYPE2 = 1
```
Для ограничения на уровень дохода BC (средний и выше среднего) нам потребуется переменная INCOME_GROUPS
```
INCOME_GROUPS IN (2, 3)
```
Объединим выражения через оператор AND
```
CITY_TYPE2 = 1 AND INCOME_GROUPS IN (2, 3)
```

### Задаем параметры расчета

In [ ]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Формируем задание на расчет

В demo_filter передаем сформированный синтаксиc переменной в кавычках

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'All700+BC'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = 'site = 101'

# задаем социально-демографическую группу в demo_filter
demo_filter = "CITY_TYPE2 = 1 AND INCOME_GROUPS IN (2, 3)"

# задаем список статистик для расчета
statistics = ["UnwReach"]

# указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
}

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total